# Nanodegree Engenheiro de Machine Learning
## Aprendizagem Supervisionada
## Project 2: Construindo um Sistema de Intervenção para Estudantes

Bem-vindo ao segundo projeto do Nanodegree de Machine Learning! Neste Notebook, alguns templates de código já foram fornecidos, e será o seu trabalho implementar funcionalidades necessárias para completar este projeto com êxito. Seções que começam com **'Implementação'** no cabeçalho indicam que o bloco de código que se segue precisará de funcionalidades adicionais que você deve fornecer. Instruções serão providenciadas para cada seção e as especificações para cada implementação estarão marcadas no bloco de código com o comando `'TODO'`. Tenha certeza de ler atentamente todas as instruções!

Além do código implementado, haverá questões relacionadas ao projeto e à implementação que você deve responder. Cada seção em que você tem que responder uma questão será antecedida de um cabeçalho **'Questão X'**. Leia atentamente cada questão e escreva respostas completas nas caixas de texto subsequentes que começam com **'Resposta: '**. O projeto enviado será avaliado baseado nas respostas para cada questão e a implementação que você forneceu.  

>**Nota:** Células de código e Markdown podem ser executadas utilizando o atalho de teclado **Shift + Enter**. Além disso, as células Markdown podem ser editadas, um clique duplo na célula entra no modo de edição.

### Questão 1 - Classificação versus Regressão
*Seu objetivo neste projeto é identificar estudantes que possam precisar de intervenção antecipada antes de serem reprovados. Que tipo de problema de aprendizagem supervisionada é esse: classificação ou regressão? Por quê?*

**Resposta: **

Classificação, pois o objetivo é prever quais alunos precisam de intervenção antes de serem reprovados, ou seja estamos classificando dois grupos de estudantes, os que precisam de intervenção, e os que não precisam. Emquanto que regressão utilizamos quando queremos descobrir um valor contínuo.

## Observando os Dados
Execute a célula de código abaixo para carregar as bibliotecas de Python necessárias e os dados sobre os estudantes. Note que a última coluna desse conjunto de dados, `'passed'`, será nosso rótulo alvo (se o aluno foi ou não aprovado). As outras colunas são atributos sobre cada aluno.

In [105]:
# Importar bibliotecas
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Ler os dados dos estudantes
student_data = pd.read_csv("student-data.csv")
print "Os dados dos estudantes foram lidos com êxito!"

student_data.head(3)

Os dados dos estudantes foram lidos com êxito!


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes


### Implementação: Observando os Dados
Vamos começar observando o conjunto de dados para determinar quantos são os estudantes sobre os quais temos informações e entender a taxa de graduação entre esses estudantes. Na célula de código abaixo, você vai precisar calcular o seguinte:
- O número total de estudantes, `n_students`.
- O número total de atributos para cada estudante, `n_features`.
- O número de estudantes aprovados, `n_passed`.
- O número de estudantes reprovados, `n_failed`.
- A taxa de graduação da classe, `grad_rate`, em porcentagem (%).


In [106]:
# TODO: Calcule o número de estudante
n_students = len(student_data.index)

# TODO: Calcule o número de atributos
# Ultima coluna é o resultado
n_features = len(student_data.columns) - 1

# TODO: Calcule o número de alunos aprovados
n_passed = len(student_data.loc[student_data['passed'] == 'yes'])

# TODO: Calcule o número de alunos reprovados
n_failed = len(student_data.loc[student_data['passed'] == 'no'])

# TODO: Calcule a taxa de graduação
grad_rate = (n_passed / float(n_students)) * 100

# Imprima os resultados
print "Número total de estudantes: {}".format(n_students)
print "Número de atributos: {}".format(n_features)
print "Número de estudantes aprovados: {}".format(n_passed)
print "Número de estudantes reprovados: {}".format(n_failed)
print "Taxa de graduação: {:.2f}%".format(grad_rate)

Número total de estudantes: 395
Número de atributos: 30
Número de estudantes aprovados: 265
Número de estudantes reprovados: 130
Taxa de graduação: 67.09%


## Preparando os Dados
Nesta seção, vamos preparara os dados para modelagem, treinamento e teste.

### Identificar atributos e variáveis-alvo
É comum que os dados que você obteve contenham atributos não numéricos. Isso pode ser um problema, dado que a maioria dos algoritmos de machine learning esperam dados númericos para operar cálculos.

Execute a célula de código abaixo para separar os dados dos estudantes em atributos e variáveis-alvo e verificar se algum desses atributos é não numérico.

In [107]:
# Extraia as colunas dos atributo
feature_cols = list(student_data.columns[:-1])

# Extraia a coluna-alvo, 'passed'
target_col = student_data.columns[-1] 

# Mostre a lista de colunas
print "Colunas de atributos:\n{}".format(feature_cols)
print "\nColuna-alvo: {}".format(target_col)

# Separe os dados em atributos e variáveis-alvo (X_all e y_all, respectivamente)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Mostre os atributos imprimindo as cinco primeiras linhas
print "\nFeature values:"
print X_all.head()

Colunas de atributos:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Coluna-alvo: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Pré-processar Colunas de Atributo

Como você pode ver, há muitas colunas não numéricas que precisam ser convertidas! Muitas delas são simplesmente `yes`/`no`, por exemplo, a coluna `internet`. É razoável converter essas variáveis em valores (binários) `1`/`0`.

Outras colunas, como `Mjob` e `Fjob`, têm mais do que dois valores e são conhecidas como variáveis categóricas. A maneira recomendada de lidar com esse tipo de coluna é criar uma quantidade de colunas proporcional aos possíveis valores (por exemplo, `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc), e assinalar `1` para um deles e `0` para todos os outros.

Essas colunas geradas são por vezes chamadas de _variáveis postiças_ (em inglês: _dummy variables_), e nós iremos utilizar a função [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) para fazer essa conversão. Execute a célula de código abaixo para executar a rotina de pré-processamento discutida nesta seção.

In [108]:
def preprocess_features(X):
    ''' Pré-processa os dados dos estudantes e converte as variáveis binárias não numéricas em
        variáveis binárias (0/1). Converte variáveis categóricas em variáveis postiças. '''
    
    # Inicialize nova saída DataFrame
    output = pd.DataFrame(index = X.index)

    # Observe os dados em cada coluna de atributos 
    for col, col_data in X.iteritems():
        
        # Se o tipo de dado for não numérico, substitua todos os valores yes/no por 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # Se o tipo de dado for categórico, converta-o para uma variável dummy
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Reúna as colunas revisadas
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))
print X_all.head(3)

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
   school_GP  school_MS  sex_F  sex_M  age  address_R  address_U  famsize_GT3  \
0          1          0      1      0   18          0          1            1   
1          1          0      1      0   17          0          1            1   
2          1          0      1      0   15          0          1            0   


### Implementação: Divisão dos Dados de Treinamento e Teste
Até agora, nós convertemos todos os atributos _categóricos_ em valores numéricos. Para o próximo passo, vamos dividir os dados (tanto atributos como os rótulos correspondentes) em conjuntos de treinamento e teste. Na célula de código abaixo, você irá precisar implementar o seguinte:
- Embaralhe aleatoriamente os dados (`X_all`, `y_all`) em subconjuntos de treinamento e teste.
  - Utilizar 300 pontos de treinamento (aproxidamente 75%) e 95 pontos de teste (aproximadamente 25%).
  - Estabelecer um `random_state` para as funções que você utiliza, se a opção existir.
  - Armazene os resultados em `X_train`, `X_test`, `y_train` e `y_test`.

In [109]:
# TODO: Importe qualquer funcionalidade adicional de que você possa precisar aqui
from sklearn.cross_validation import train_test_split

# TODO: Estabeleça o número de pontos de treinamento
num_train = 300

# Estabeleça o número de pontos de teste
num_test = n_students - num_train

# TODO: Emabaralhe e distribua o conjunto de dados de acordo com o número de pontos de treinamento e teste abaixo
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=10)

# Mostre o resultado da distribuição
print "O conjunto de treinamento tem {} amostras.".format(X_train.shape[0])
print "O conjunto de teste tem {} amostras.".format(X_test.shape[0])

O conjunto de treinamento tem 300 amostras.
O conjunto de teste tem 95 amostras.


## Treinando e Avaliando Modelos
Nesta seção, você irá escolher 3 modelos de aprendizagem supervisionada que sejam apropriados para esse problema e que estejam disponíveis no `scikit-learn`. Primeiro você irá discutir o raciocínio por trás da escolha desses três modelos considerando suas vantagens e desvantagens e o que você sabe sobre os dados. Depois você irá ajustar o modelo a diferentes tamanhos de conjuntos de treinamento (com 100, 200 e 300 pontos) e medir a pontuação F<sub>1</sub>. Você vai precisar preencher três tabelas (uma para cada modelo) que mostrem o tamanho do conjunto de treinamento, o tempo de treinamento, o tempo de previsão e a pontuação F<sub>1</sub> no conjunto de treinamento.

**Os seguintes modelos de aprendizagem supervisionada estão atualmente disponíveis no **[`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html)** para você escolher:**
- Gaussian Naive Bayes (GaussianNB)
- Árvores de Decisão
- Métodos de agregação (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Método do gradiente estocástico (SGDC)
- Máquinas de vetores de suporte (SVM)
- Regressão logística

### Questão 2 - Aplicação dos Modelos
*Liste três modelos de aprendizagem supervisionada que são apropriadas para esse problema. Para cada modelo escolhido:*
- Descreva uma aplicação em mundo real na indústria em que o modelo pode ser aplicado. *(Talvez você precise fazer um pouco de pesquisa para responder essa questão – dê as devidas referências!)* 
- Quais são as vantagens do modelo; quando ele tem desempenho melhor? 
- Quais são as desvantagens do modelo, quando ele tem desempenho pior?
- O que faz desse modelo um bom candidato para o problema, considerando o que você sabe sobre os dados?

**Resposta: **

**Classificador 1 - Model A: Logistic Regression: **

*Aplicação Real:*<br/>
Previsões na área médica onde o resultado é booleano, se a pessoa "está" ou "não está" apresentando depressão.<br/>
Refêrencia: Quora https://www.quora.com/What-are-applications-of-linear-and-logistic-regression Acessado em 22/12/2017.<br/>

*Vantagens:*<br/>
-Treino rápido;<br/>
-Modelo linear;<br/>
-Baixa Variância;<br/>

*Desvantagens:*<br/>
-Sobreajusta o modelo com facilidade;<br/>
-Alto viés;<br/>

*Neste Problema:*
Este modelo pode apresentar um bom resultado para este problema, pois os dados foram bem ajustados para um modelo linear (dados categoricos, foram facilmente ajustados para valores numéricos, como por exemplo transformamos "sim" e "nao" em 1 e 0). E, baseado na referências da aplicação real, a regressção logística é boa para classificar resultados que possuem duas classes de saída, exemplo: "aprovados" e "não aprovados".


**Classificador 2 - Model B: Decision Tree Classifier: **

*Aplicação Real:*<br/>
Gerenciamento na área de negócios, devido a facilidade de interpretação gráfica, e de utilizar features que são "comprensíveis", os algoritmos de árvore de decisão são geralmente usados em tomadas de decisão na área de negócios.<br/>
Refêrencia: What-When-How http://what-when-how.com/artificial-intelligence/decision-tree-applications-for-data-modelling-artificial-intelligence/ Acessado em 22/12/2017.<br/>

*Vantagens:*<br/>
-Trata com facilidade variáveis categoricas;<br/>
-Fácil interpretação gráfica;<br/>

*Desvantagens:*<br/>
-Tende a sobreajustar o modelo;<br/>

*Neste Problema:*
Este modelo pode ter um resultado satisfatório neste problema, pois se assemelha com os problemas de gerenciamento na área de negócios, e os dados se encaixam bem com as vantagens deste modelo.

**Classificador 3 - Model C: Support Vector Machine: **

*Aplicação Real:*<br/>
SVM são muito utilizados em classificação de imagens, entre elas detecção de rostos.<br/>
Refêrencia: Data-flair https://data-flair.training/blogs/applications-of-svm/ Acessado em 22/12/2017.<br/>

*Vantagens:*<br/>
-Funciona bem em conjuntos pequenos com dados limpos e organizados;<br/>
-Preciso e eficiente pois usa subconjuntos de pontos de treinamento;<br/>
*Desvantagens:*<br/>
-Baixo desempenho em grandes conjuntos de dados;<br/>
-Eficiência cai com conjuntos de dados muito desorganizados(ou dados muito fora do desvio padrão, etc);<br/>

*Neste Problema:*
É esperado que neste problema o classificador SVM tenha uma boa eficiência, e um desempenho bom/médio, pois estamos trabalhando com um conjunto pequeno, de aproximadamente 360 registros e com 30 colunas, os dados também já foram recebidos organizados e limpos. Ou seja, o conjunto de dados deste problema se encaixa bem nos prós deste modelo.


### Configuração
Execute a célula de código abaixo para inicializar três funções de ajuda que você pode utilizar para treinar e testar os três modelos de aprendizagem supervisionada que você escolheu acima. As funções são as seguintes:
- `train_classifier` - recebe como parâmetro um classificador e dados de treinamento e ajusta o classificador aos dados.
- `predict_labels` - recebe como parâmetro um classificador ajustado, atributos e rótulo alvo e faz estimativas utilizando a pontuação do F<sub>1</sub>.
- `train_predict` - recebe como entrada um classificador, e dados de treinamento e teste, e executa `train_clasifier` e `predict_labels`.
 - Essa função vai dar a pontuação F<sub>1</sub> tanto para os dados de treinamento como para os de teste, separadamente.

In [110]:
#implemtação de codigo para imprimir o markdown sozinho :)

global col_1
global col_2
global col_3
global col_4
global col_5
col_1 = 'Tamanho do Conj de Treinamento'
col_2 = 'Tempo de Treinamento'
col_3 = 'Tempo de Estimativa (teste)'
col_4 = 'Pontuacao F1 (treinamento)'
col_5 = 'Pontuacao F1 (teste)'

global df_model
df_model = pd.DataFrame(columns=[col_1, col_2, col_3, col_4, col_5], index=[100, 200, 300])
df_backup = df_model.copy()


def pandas_df_to_markdown_table(df):
    '''
    Função para tabular com markdown.
    Creditos para o autor na publicação abaixo.
    Acessado em https://stackoverflow.com/questions/33181846/programmatically-convert-pandas-dataframe-to-markdown-table
    '''
    from IPython.display import Markdown, display
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    display(Markdown(df_formatted.to_csv(sep="|", index=False)))

In [111]:
def train_classifier(clf, X_train, y_train, index=None):
    ''' Ajusta um classificador para os dados de treinamento. '''
    
    # Inicia o relógio, treina o classificador e, então, para o relógio
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Imprime os resultados
    print "O modelo foi treinado em {:.4f} segundos".format(end - start)
    if index:
        df_model.loc[index,col_2] = "{:.4f} seg".format(end - start)

    
def predict_labels(clf, features, target, index=None):
    ''' Faz uma estimativa utilizando um classificador ajustado baseado na pontuação F1. '''
    
    # Inicia o relógio, faz estimativas e, então, o relógio para
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Imprime os resultados de retorno
    print "As previsões foram feitas em {:.4f} segundos.".format(end - start)
    if index:
        df_model.loc[index, col_3] = "{:.4f} seg".format(end - start)
    
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Treina e faz estimativas utilizando um classificador baseado na pontuação do F1. '''
    
    index = len(X_train)
    
    # Indica o tamanho do classificador e do conjunto de treinamento
    print "Treinando um {} com {} pontos de treinamento. . .".format(clf.__class__.__name__, index)
    df_model.loc[index, col_1] = index
    
    # Treina o classificador
    train_classifier(clf, X_train, y_train, index)
    
    # Imprime os resultados das estimativas de ambos treinamento e teste
    time_train = predict_labels(clf, X_train, y_train, index)
    time_test = predict_labels(clf, X_test, y_test, index)
    print "Pontuação F1 para o conjunto de treino: {:.4f}.".format(time_train)
    print "Pontuação F1 para o conjunto de teste: {:.4f}.".format(time_test)
    df_model.loc[index, col_4] = "{:.4f}.".format(time_train)
    df_model.loc[index, col_5] = "{:.4f}.".format(time_test)

### Implementação: Métricas de Desempenho do Modelo
Com as funções acima, você vai importar os três modelos de aprendizagem supervisionada de sua escolha e executar a função `train_prediction` para cada um deles. Lembre-se de que você vai precisar treinar e usar cada classificador para três diferentes tamanhos de conjuntos de treinamentos: 100, 200 e 300 pontos. Então você deve ter 9 saídas diferentes abaixo – 3 para cada modelo utilizando cada tamanho de conjunto de treinamento. Na célula de código a seguir, você deve implementar o seguinte:
- Importe os três modelos de aprendizagem supervisionada que você escolheu na seção anterior.
- Inicialize os três modelos e armazene eles em `clf_A`, `clf_B` e `clf_C`.
 - Defina um `random_state` para cada modelo, se a opção existir.
 - **Nota:** Utilize as configurações padrão para cada modelo – você vai calibrar um modelo específico em uma seção posterior.
- Crie diferentes tamanhos de conjuntos de treinamento para treinar cada modelo.
 - *Não embaralhe e distribua novamente os dados! Os novos pontos de treinamento devem ser tirados de `X_train` e `y_train`.*
- Treine cada modelo com cada tamanho de conjunto de treinamento e faça estimativas com o conjunto de teste (9 vezes no total).  
**Nota:** Três tabelas são fornecidas depois da célula de código a seguir, nas quais você deve anotar seus resultados.

In [112]:
# TODO: Importe os três modelos de aprendizagem supervisionada do sklearn
# from sklearn import model_A
from sklearn.linear_model import LogisticRegression
# from sklearn import model_B
from sklearn.tree import DecisionTreeClassifier
# from skearln import model_C
from sklearn.svm import SVC

# definição de um random state 'fixo'
custom_random_state = 32

# TODO: Inicialize os três modelos
clf_A = LogisticRegression(random_state=custom_random_state)
clf_B = DecisionTreeClassifier(random_state=custom_random_state)
clf_C = SVC(random_state=custom_random_state)

# Cria array de classificadores e dataframe de resultados
clfs = [clf_A, clf_B, clf_C]
df_A = None
df_B = None
df_C = None
dfs = [df_A, df_B, df_C]

# TODO: Configure os tamanho dos conjuntos de treinamento
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

train_sets = [(X_train_100, y_train_100), (X_train_200, y_train_200), (X_train_300, y_train_300)]

# TODO: Executar a função 'train_predict' para cada classificador e cada tamanho de conjunto de treinamento
# train_predict(clf, X_train, y_train, X_test, y_test)
for i, x in enumerate(clfs):
    for current_X, current_y in train_sets:
        train_predict(clfs[i], current_X, current_y, X_test, y_test)
        print ''
    dfs[i] = df_model.copy()
    df_model = df_backup.copy()

Treinando um LogisticRegression com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0030 segundos
As previsões foram feitas em 0.0010 segundos.
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 0.8593.
Pontuação F1 para o conjunto de teste: 0.7612.

Treinando um LogisticRegression com 200 pontos de treinamento. . .
O modelo foi treinado em 0.0040 segundos
As previsões foram feitas em 0.0010 segundos.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de treino: 0.8444.
Pontuação F1 para o conjunto de teste: 0.7591.

Treinando um LogisticRegression com 300 pontos de treinamento. . .
O modelo foi treinado em 0.0050 segundos
As previsões foram feitas em 0.0000 segundos.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de treino: 0.8263.
Pontuação F1 para o conjunto de teste: 0.8169.

Treinando um DecisionTreeClassifier com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0000 segundos


### Resultados Tabulados
Edite a célula abaixo e veja como a tabela pode ser desenhada em [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). Você deve salvar seus resultados abaixo nas tabelas fornecidas.

In [113]:
#Exibindo tabelas automaticamente
for i, x in enumerate(clfs):
    print '\033[1m' + '\033[4m' + "\nClassificador " + str(i+1) + ' - ' + clfs[i].__class__.__name__ + '\033[0m'
    pandas_df_to_markdown_table(dfs[i])


Classificador 1 - LogisticRegression


Tamanho do Conj de Treinamento|Tempo de Treinamento|Tempo de Estimativa (teste)|Pontuacao F1 (treinamento)|Pontuacao F1 (teste)
---|---|---|---|---
100|0.0030 seg|0.0010 seg|0.8593.|0.7612.
200|0.0040 seg|0.0000 seg|0.8444.|0.7591.
300|0.0050 seg|0.0000 seg|0.8263.|0.8169.



Classificador 2 - DecisionTreeClassifier


Tamanho do Conj de Treinamento|Tempo de Treinamento|Tempo de Estimativa (teste)|Pontuacao F1 (treinamento)|Pontuacao F1 (teste)
---|---|---|---|---
100|0.0000 seg|0.0000 seg|1.0000.|0.6667.
200|0.0000 seg|0.0000 seg|1.0000.|0.6818.
300|0.0150 seg|0.0000 seg|1.0000.|0.6774.



Classificador 3 - SVC


Tamanho do Conj de Treinamento|Tempo de Treinamento|Tempo de Estimativa (teste)|Pontuacao F1 (treinamento)|Pontuacao F1 (teste)
---|---|---|---|---
100|0.0160 seg|0.0000 seg|0.8366.|0.8228.
200|0.0150 seg|0.0000 seg|0.8552.|0.7947.
300|0.0000 seg|0.0000 seg|0.8615.|0.8079.




## Escolhendo o Melhor Modelo
Nesta seção final, você irá escolher dos três modelos de aprendizagem supervisionada o *melhor* para utilizar os dados dos estudantes. Você então executará um busca em matriz otimizada para o modelo em todo o conjunto de treinamento (`X_train` e `y_train`) ao calibrar pelo menos um parâmetro, melhorando em comparação a pontuação F<sub>1</sub> do modelo não calibrado. 

### Questão 3 - Escolhendo o Melhor Modelo
*Baseando-se nos experimentos que você executou até agora, explique em um ou dois parágrafos ao conselho de supervisores qual modelo que você escolheu como o melhor. Qual modelo é o mais apropriado baseado nos dados disponíveis, recursos limitados, custo e desempenho?*

**Resposta: **

Segundo os resultados obtidos e tabulados, o melhor classificador para este caso de uso é o Supprt Vector Machine. Justificativas:

Como previsto o classificador por árvore de decisão se mostrou sobreajustado, em diferentes conjuntos de treinamento, a pontuação f1 de treinamento esteve sempre em 1.0 (ver tabela abaixo), o que demonstra que o modelo sobreajustou para atender os pontos de treinamento, e por isso ele não conseguiu ser genérico o suficiente para classificar os dados de teste, obtendo a pior pontuação f1 de teste entre os modelos, o que eliminou a árvore de decisão já neste quesito.

In [114]:
print '\033[1m' + '\033[4m' + "\nClassificador 2" + ' - ' + clfs[1].__class__.__name__ + '\033[0m'
pandas_df_to_markdown_table(dfs[1][[col_1, col_4, col_5]].copy())


Classificador 2 - DecisionTreeClassifier


Tamanho do Conj de Treinamento|Pontuacao F1 (treinamento)|Pontuacao F1 (teste)
---|---|---
100|1.0000.|0.6667.
200|1.0000.|0.6818.
300|1.0000.|0.6774.


Já a decisão entre Regressão Logística e Support Vector Machine foi muito difícil, pois ambos os modelos foram rodados em diferentes "random_state" e em cada momento um se saia melhor que o outro (no que diz respeito a pontuação f1), no entanto, o Regressor Logístico se mostrou mais estável, e possui um desempenho de processamento duas vezes mais rápido que o SVM, tanto do treinamento, quanto no teste. Além disso, a pontuação f1 de treino está bem próxima da de teste, o que demonstra que o modelo está bem ajustado para este projeto. Portanto, o modelo escolhido foi o de Regressão Logística, e o melhor desempenho foi com um conjunto de treinamento de 300 elementos, se mostrando um dos modelos com mais rápido processamento, e com melhor pontuação f1.

In [115]:
print '\033[1m' + '\033[4m' + "\nMelhor Modelo: " + clfs[0].__class__.__name__ + '\033[0m'
pandas_df_to_markdown_table(dfs[0].loc[[300]])


Melhor Modelo: LogisticRegression


Tamanho do Conj de Treinamento|Tempo de Treinamento|Tempo de Estimativa (teste)|Pontuacao F1 (treinamento)|Pontuacao F1 (teste)
---|---|---|---|---
300|0.0050 seg|0.0000 seg|0.8263.|0.8169.


### Questão 4 – O Modelo para um Leigo
*Em um ou dois parágrafos, explique para o conselho de supervisores, utilizando termos leigos, como o modelo final escolhido deve trabalhar. Tenha certeza que você esteja descrevendo as melhores qualidades do modelo, por exemplo, como o modelo é treinado e como ele faz uma estimativa. Evite jargões técnicos ou matemáticos, como descrever equações ou discutir a implementação do algoritmo.*

**Resposta: **

Nosso modelo trabalha em duas etapas:

Na primeira o modelo aprende com os dados levantados na tabela de atributos, e o indicador se o aluno irá se formar ou não. O modelo consegue fazer associações, atributo por atributo, de cada aluno, e sabendo se ele se formou ou não, consegue identificar pesos para cada um e assim prever a aprovação ou repetência de um aluno. 

A segunda etapa, com o modelo já aprendido, é prever se uma lista de novos alunos, com os mesmos atributos, quais conseguirão se formar sem auxílio extra, e identificando aqueles que podem reprovar podemos então tomar medidas preventivas. O modelo pode fazer isso, pois compara cada atributo dos novos alunos com os atributos e o peso identificado anteriormente (com os dados de alunos que foram aprovados ou reprovados no passado). Algumas vantagens desse método de identificação são: É um modelo de ágil processamento e os atributos dos alunos podem ser isolados e independentes.

### Implementação: Calibrando o Modelo
Calibre o modelo escolhido. Utilize busca em matriz (`GridSearchCV`) com, pelo menos, um parâmetro importante calibrado com, pelo menos, 3 valores diferentes. Você vai precisar utilizar todo o conjunto de treinamento para isso. Na célula de código abaixo, você deve implementar o seguinte:
- Importe [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) e [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Crie o dicionário de parâmetros que você deseja calibrar para o modelo escolhido.
 - Examplo: `parameters = {'parameter' : [list of values]}`.
- Inicialize o classificador que você escolheu e armazene-o em `clf`.
- Crie a função de pontuação F<sub>1</sub> utilizando `make_scorer` e armazene-o em `f1_scorer`.
 - Estabeleça o parâmetro `pos_label` para o valor correto!
- Execute uma busca em matriz no classificador `clf` utilizando o `f1_scorer` como método de pontuação e armazene-o em `grid_obj`.
- Treine o objeto de busca em matriz com os dados de treinamento (`X_train`, `y_train`) e armazene-o em `grid_obj`.

In [116]:
# TODO: Importe 'GridSearchCV' e 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer

# Novo embaralhamento para busca de parametros
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_all, y_all, test_size=100, random_state=30)

# TODO: Crie a lista de parâmetros que você gostaria de calibrar
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
solver = ['sag']
class_weight = ['balanced', None]
parameters = {'C': C,
              'solver': solver,
              'class_weight': class_weight}

# TODO: Inicialize o classificador
clf = LogisticRegression(random_state=custom_random_state)

# TODO: Faça uma função de pontuação f1 utilizando 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Execute uma busca em matriz no classificador utilizando o f1_scorer como método de pontuação
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer, cv=5, verbose=0)

# TODO: Ajuste o objeto de busca em matriz para o treinamento de dados e encontre os parâmetros ótimos
grid_obj.fit(X_train_cv, y_train_cv)

# Get the estimator
clf = grid_obj.best_estimator_

# Reporte a pontuação final F1 para treinamento e teste depois de calibrar os parâmetros
tuned_f1_train = "{:.4f}".format(predict_labels(clf, X_train_300, y_train_300))
tuned_f1_test = "{:.4f}".format(predict_labels(clf, X_test, y_test))
print "\nO modelo calibrado tem F1 de " + tuned_f1_train + " no conjunto de treinamento."
print "O modelo calibrado tem F1 de " + tuned_f1_test + " no conjunto de teste."

print "\nMelhores parametros:"
print "C = " + str(grid_obj.best_params_['C'])
print "class_weight = " + str(grid_obj.best_params_['class_weight'])
print "solver = " + str(grid_obj.best_params_['solver'])

df_tuned = dfs[0].copy()
df_tuned.loc[300, col_4] = tuned_f1_train
df_tuned.loc[300, col_5] = tuned_f1_test

As previsões foram feitas em 0.0010 segundos.
As previsões foram feitas em 0.0010 segundos.

O modelo calibrado tem F1 de 0.8267 no conjunto de treinamento.
O modelo calibrado tem F1 de 0.8243 no conjunto de teste.

Melhores parametros:
C = 0.1
class_weight = None
solver = sag


### Questão 5 - Pontuação F<sub>1</sub> Final
*Qual é a pontuação F<sub>1</sub> do modelo final para treinamento e teste? Como ele se compara ao modelo que não foi calibrado?*

**Resposta: **

Primeiramente, segui esta referência e fiz um novo embaralhamento de dados para buscar os melhores parâmetros.<br/>
Referência: https://stackoverflow.com/questions/43530761/cross-validation-with-grid-search-returns-worse-results-than-default Acessado em 23/12/2017. <br/>
Depois eu apliquei a comparação sobre o mesmo modelo que defini como sendo o melhor, que no caso foi o de Regressão Logística com 300 elementos. Assim obtive os seguintes resultados:<br/>


In [117]:
print '\033[1m' + '\033[4m' + "\nAntes: " + clfs[0].__class__.__name__ + '\033[0m'
pandas_df_to_markdown_table(dfs[0].loc[[300],[col_1, col_4, col_5]])

print '\033[1m' + '\033[4m' + "\nDepois: " + clfs[0].__class__.__name__ + '\033[0m'
pandas_df_to_markdown_table(df_tuned.loc[[300],[col_1, col_4, col_5]])


Antes: LogisticRegression


Tamanho do Conj de Treinamento|Pontuacao F1 (treinamento)|Pontuacao F1 (teste)
---|---|---
300|0.8263.|0.8169.



Depois: LogisticRegression


Tamanho do Conj de Treinamento|Pontuacao F1 (treinamento)|Pontuacao F1 (teste)
---|---|---
300|0.8267|0.8243


Como podemos notar, o modelo teve um ligeira melhora nos resultados, tanto de treinamento, quanto de teste, assim como a distância entre as duas pontuações diminuiu, o que tornou o modelo ainda mais bem ajustado ao projeto, reforçando a boa escolha do classificador para o problema dado.

> **Nota**: Uma vez que você completou todas as implementações de código e respondeu todas as questões acima com êxito, você pode finalizar seu trabalho exportando o iPython Nothebook como um document HTML. Você pode fazer isso utilizando o menu acima e navegando para  
**File -> Download as -> HTML (.html)**. Inclua a documentação final junto com o notebook para o envio do seu projeto.